In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F 
import numpy 
import transformers
import tiktoken
from transformers import AutoTokenizer

/home/auschra/ml/llm/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
torch.manual_seed(1337)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'


In [3]:
"""todos
    EX1: The n-dimensional tensor mastery challenge: Combine the `Head` and `MultiHeadAttention`
     into one class that processes all the heads in parallel, treating the heads as another batch dimension (answer is in nanoGPT).
     
    EX2: Train the GPT on your own dataset of choice! What other data could be fun to blabber on about? (A fun advanced suggestion 

    if you like: train a GPT to do addition of two numbers, i.e. a+b=c. You may find it helpful to predict the digits of c in reverse
     order, as the typical addition algorithm (that you're hoping it learns) would proceed right to left too. You may want to modify 
     the data loader to simply serve random problems and skip the generation of train.bin, val.bin. You may want to mask out the loss 
     at the input positions of a+b that just specify the problem using y=-1 in the targets (see CrossEntropyLoss ignore_index). Does 
     your Transformer learn to add? Once you have this, swole doge project: build a calculator clone in GPT, for all of +-*/. Not an easy problem. You may need Chain of Thought traces.)

    EX3: Find a dataset that is very large, so large that you can't see a gap between train and val loss. Pretrain the transformer 
    on this data, then initialize with that model and finetune it on tiny shakespeare with a smaller number of steps and lower learning
     rate. Can you obtain a lower validation loss by the use of pretraining?
     
    EX4: Read some transformer papers and implement one additional feature or change that people seem to use. Does it improve the performance of your GPT?
"""

"todos\n    EX1: The n-dimensional tensor mastery challenge: Combine the `Head` and `MultiHeadAttention`\n     into one class that processes all the heads in parallel, treating the heads as another batch dimension (answer is in nanoGPT).\n     \n    EX2: Train the GPT on your own dataset of choice! What other data could be fun to blabber on about? (A fun advanced suggestion \n\n    if you like: train a GPT to do addition of two numbers, i.e. a+b=c. You may find it helpful to predict the digits of c in reverse\n     order, as the typical addition algorithm (that you're hoping it learns) would proceed right to left too. You may want to modify \n     the data loader to simply serve random problems and skip the generation of train.bin, val.bin. You may want to mask out the loss \n     at the input positions of a+b that just specify the problem using y=-1 in the targets (see CrossEntropyLoss ignore_index). Does \n     your Transformer learn to add? Once you have this, swole doge project: bu

In [4]:
# params
# B, T, C
batch_size = 64
block_size = 64
n_embd = 256
head_size = 64
learning_rate =1e-4
n_layer = 4
n_head = 6
dropout = 0.2
max_iters = 500


In [5]:
with open('shakespeare.txt', 'r') as f:
    data = f.read()

# encoding 
enc = tiktoken.get_encoding('cl100k_base')
data = enc.encode(data)

# remap openai tokens to only represent present tokens in input
unique_toks = sorted(set(data))


#token_id_map = {old_id: new_id for new_id, old_id in enumerate(unique_toks)}
#remapped_data = [token_id_map[token_id] for token_id in encoded_data]
vocab_size = 100252

n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

train_data = torch.tensor(train_data, dtype=torch.long)
val_data = torch.tensor(val_data, dtype=torch.long)


In [6]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))     # get batch_size starting points
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])

    return x, y

xb, yb = get_batch('train')

print(xb.shape)
print(yb.shape)

torch.Size([64, 64])
torch.Size([64, 64])


In [7]:
class LayerNorm1D:
    # normalise each batch -> that way to get diff from mean and unit variance
    def __init__(self, dim, eps=1e-5, momentum=0.1): # params
        self.eps = eps
        self.gamma = torch.ones(dim)    
        self.beta = torch.zeros(dim)

    def __call__(self, x):
        xmean = x.mean(1, keepdim= True)    # batch mean
        xvar = x.var(1, keepdim=True)          # batch var
        xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalise a row, columwise 
        self.out = self.gamma * xhat + self. beta
        return self.out
        
    def parameters(self):       
        return [self.gamma, self.beta]

module = LayerNorm1D(100)   # 100 dimensions
x = torch.randn(32, 100)    # 32 batch, 100 dims

x = module(x)
x.shape

torch.Size([32, 100])

In [8]:
class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()

        # linear layers
        self.key = nn.Linear(n_embd, head_size, bias=False)      # (B, T, head_size)
        self.query = nn.Linear(n_embd, head_size, bias=False)   # (B, T, head_size)
        self.value = nn.Linear(n_embd, head_size, bias=False)   # (B, T, head_size)
        self.register_buffer('tril', torch.ones(block_size, block_size))    # (B, T, T)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x) # B, T, head_size   
        q = self.key(x) # B, T, head_size 
        


        tril = self.tril[:T, :T].to(device)    

        # attention scores
        wei = q @ k.transpose(-2, -1) #keepdim=True) # (B, T, head_size) @ (B, head_size, T) -> keep batch dim so (B, T, T)     
        wei = wei.masked_fill(tril == 0, float('-inf'))     # lower triangular matrix (B, T, T)
        wei = F.softmax(wei, dim = -1)              # normalise 
        wei = self.dropout(wei)                      # dropout

        v = self.key(x) # B, T, head_size 

        out = wei @ v            # (B, T, T) @ (B, T, C) -> (B, T, C)
        
        return out

class MultiHeadAttention(nn.Module):
    def __init__(self, n_head, head_size):
        super().__init__()
        self.heads = nn.ModuleList(Head(head_size) for _ in range(n_head))
        self.proj = nn.Linear(n_head * head_size, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)     # (B, T, C) # each head_size gets added together
        out = self.dropout(self.proj(out))
        return out

In [9]:
# combine into causal self attention

class CausalSelfAttention(nn.Module):
    def __init___(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList()  # B, T, head_size, num_heads


In [10]:
class FeedForward(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(n_embd, 4*n_embd),
                                nn.ReLU(),
                                nn.Linear(4*n_embd, n_embd),
                                nn.Dropout(dropout),)

    def forward(self, x):
        return self.net(x)

In [11]:
class Block(nn.Module):
    # transformer block, attention -> mlp
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)    # create this many heads, fits into n_ebd
        self.ffd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffd(self.ln2(x))

        return x

In [12]:
class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)       # final layer norm after mlp?
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding_table(idx) # embed x     # B, T, C
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))   # T, C
        x = tok_emb + pos_emb # B, T, C 
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is B, T indices of context, unembedded
        idx = idx.to(next(self.parameters()).device)
        for _ in range(max_new_tokens):
            # get block size of last input
            idx_cond = idx[:, -block_size:]
            # get predicitons
            logits, _ = self(idx_cond)   
            
            # last time step?
            logits = logits[:, -1, :] # (B, C)
            probs = F.softmax(logits, dim=-1)
            # sample from dist
            idx_next = torch.multinomial(probs, num_samples = 1) # B, 1
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            
            return idx

In [13]:
model = GPT().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):

    # evaluate the loss on train / val every so often
    xb, yb = get_batch('train')
    xb = xb.to(device)
    yb = yb.to(device)

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    if iter % 100 == 0:
        print(f'Iter {iter}, loss: {loss.item()}')


model.eval()

# Generate text without tracking gradients
with torch.no_grad():
    context = torch.zeros((1,1), dtype=torch.long).to(device)
    y = model.generate(context, 100)
    text = y[0].tolist()
    text = enc.decode(text)
    print(text)



Iter 0, loss: 11.700955390930176


../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [0,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [1,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [2,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [3,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [4,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [5,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread:

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


ad: [95,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [96,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [97,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [98,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [99,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [100,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/cuda/Indexing.cu:1308: indexSelectLargeIndex: block: [64,0,0], thread: [101,0,0] Assertion `srcIndex < srcSelectDimSize` failed.
../aten/src/ATen/native/c